In [1]:
import xgboost
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

import numpy as np
import math

from pickle import dump

import datetime
# import timedelta
# 오늘날짜 기준
dt = datetime.datetime.now()
fl_dt = float(str(dt.year) + "." + str(dt.month))

In [7]:
## 지하철역 거리 가져오기 추가
import json
from pandas.io.json import json_normalize

db_con = pymysql.connect(
    user='zip_researcher', 
    passwd='zd_LyowVGd9TyvACeAb5z9BRjDjrGGId3mzlNJD', 
    host='zipfund-price.c8vjhowj9gyc.ap-northeast-2.rds.amazonaws.com', 
    db='kb_raw_data', 
    charset='utf8'
)

cursor = db_con.cursor(pymysql.cursors.DictCursor)

sql = """
SELECT kb_apartment_id, facility_info FROM kb_raw_data.nearby_apart_info
"""

cursor.execute(sql)
result_0 = cursor.fetchall()
result_0 = pd.DataFrame(result_0)

c_load = result_0["facility_info"].apply(json.loads)
c_list = list(c_load)
c_dat = json.dumps(c_list)

result_0 = result_0.join(pd.read_json(c_dat))


result_1 = result_0[["kb_apartment_id", "subway"]]


res_list1 = pd.DataFrame()
apt_id_list = []
distance_res_list = []

for i in result_1.index:
    
    apt_id = result_1["kb_apartment_id"][i]
    
    try :
        res = len(pd.DataFrame.from_dict(result_1["subway"][i])['distance']) + (1 / min(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int))) * 100 # 0.44 , 0.5
#         res =  len(pd.DataFrame.from_dict(result_1["subway"][i])['distance']) * (1 / min(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)))  # 0.3
#         res =  len(pd.DataFrame.from_dict(result_1["subway"][i])['distance']) * ((1 / min(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int))) * 100) # 0.3
#         res = min(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)) # 0.26,  0.3
#         res =  sum(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)) # 0.42 , 0.48
#         res =  sum(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)) / len(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)) # 0.34 , 0.39
#         res = len(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)) / (sum(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)) / len(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)))# 0.43, 0.49
#         res = len(pd.DataFrame.from_dict(result_1["subway"][i])['distance']) + (1 / sum(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int))) # 0.44, 0.5
#         res = len(pd.DataFrame.from_dict(result_1["subway"][i])['distance']) + (1 / (sum(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int)) / len(pd.DataFrame.from_dict(result_1["subway"][i])['distance']))) * 100 # 0.45, 0.5

    except :
        res = 0
        
        
    apt_id_list.append(apt_id)
    distance_res_list.append(res)
        
res_list1 = res_list1.append(list(zip(apt_id_list, distance_res_list)))
#     res_list = res_list.append(zip(apt_id, res), ignore_index=True)

res_list1.columns = ['kb_apartment_id', 'subway_index']
# res_list["subway_distance"] = res_list["subway_distance"].astype(int)
res_list1

# test = test.drop(column , axis=1)

,kb_apartment_id,subway_index
0,1,0.000000
1,2,0.000000
2,3,4.223714
3,4,4.223214
4,5,5.322581
...,...,...
22387,22543,0.000000
22388,22544,4.213220
22389,22545,7.380228
22390,22546,0.000000


In [9]:
## 지하철역 거리 가져오기 추가
import json
from pandas.io.json import json_normalize

db_con = pymysql.connect(
    user='', 
    passwd='', 
    host='', 
    db='', 
    charset=''
)

cursor = db_con.cursor(pymysql.cursors.DictCursor)

# sql = """
# SELECT * FROM
# """

cursor.execute(sql)
result_0 = cursor.fetchall()
result_0 = pd.DataFrame(result_0)

c_load = result_0["facility_info"].apply(json.loads)
c_list = list(c_load)
c_dat = json.dumps(c_list)

result_0 = result_0.join(pd.read_json(c_dat))


result_1 = result_0[["kb_apartment_id", "subway"]]


res_list3 = pd.DataFrame()
apt_id_list = []
distance_res_list = []

for i in result_1.index:
    
    apt_id = result_1["kb_apartment_id"][i]
    
    try :
        res =  max(pd.DataFrame.from_dict(result_1["subway"][i])['distance'].astype(int))

    except :
        res = 0
        
        
    apt_id_list.append(apt_id)
    distance_res_list.append(res)
        
res_list3 = res_list3.append(list(zip(apt_id_list, distance_res_list)))
#     res_list = res_list.append(zip(apt_id, res), ignore_index=True)

res_list3.columns = ['kb_apartment_id', 'max_subway_distance']
# res_list["subway_distance"] = res_list["subway_distance"].astype(int)
res_list3

# test = test.drop(column , axis=1)

,kb_apartment_id,max_subway_distance
0,1,0
1,2,0
2,3,995
3,4,994
4,5,984
...,...,...
22387,22543,0
22388,22544,551
22389,22545,870
22390,22546,0


In [44]:
# final_result_addr_all["평형구분"] = final_result_addr_all.apply(lambda x : area_jp(x['net_area']), axis = 1)

# final_result_addr_all['매매가'] = (final_result_addr_all['price_per_size'] / 3.305785) * final_result_addr_all['net_area']
# final_result_addr_all['전세가'] = (final_result_addr_all['rent_per_size'] / 3.305785) * final_result_addr_all['net_area']
final_result_addr_all['avg_trans_time'] = (final_result_addr_all['강남역_소요시간'] +  final_result_addr_all['시청역_소요시간'] +  final_result_addr_all['여의도역_소요시간']) /3

final_result_addr_all

,kb_unit_type_id,kb_apartment_id,net_area,number_of_units,rent_ratio,kb_price_growth,kb_rent_growth,x,y,number_of_parking,...,view_name,search_name,province_name,city_name,district_name,province_id,city_id,district_id,district_code,avg_trans_time
0,1,1,83.61,42,0.5529,13.33,0.00,126.968160,37.591500,126,...,청운 벽산빌리지,청운벽산빌리지,서울특별시,종로구,청운동,1,1,1,1111010100,40.000000
1,2,1,132.39,20,0.4957,21.05,0.00,126.968160,37.591500,126,...,청운 벽산빌리지,청운벽산빌리지,서울특별시,종로구,청운동,1,1,1,1111010100,40.000000
2,3,1,145.73,10,0.5000,20.00,0.00,126.968160,37.591500,126,...,청운 벽산빌리지,청운벽산빌리지,서울특별시,종로구,청운동,1,1,1,1111010100,40.000000
3,4,2,84.51,20,0.6667,2.27,3.45,126.967381,37.586360,88,...,청운 현대,청운현대,서울특별시,종로구,청운동,1,1,1,1111010100,37.333333
4,5,2,129.76,20,0.5600,0.00,0.00,126.967381,37.586360,88,...,청운 현대,청운현대,서울특별시,종로구,청운동,1,1,1,1111010100,37.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25782,66418,21007,84.98,95,0.6268,14.52,15.58,126.848279,37.371110,1219,...,시흥 목감 호반베르디움 5차,시흥목감호반베르디움5차,경기도,시흥시,산현동,8,105,2604,4139012300,80.333333
25783,66419,21008,59.51,147,0.5604,23.95,31.82,126.958711,37.500384,432,...,흑석동양,흑석동양,서울특별시,동작구,흑석동,1,20,414,1159010500,37.666667
25784,66420,21008,84.99,276,0.6118,12.86,33.03,126.958711,37.500384,432,...,흑석동양,흑석동양,서울특별시,동작구,흑석동,1,20,414,1159010500,37.666667
25785,66459,21013,59.97,181,0.6535,31.17,26.92,126.657820,37.412493,0,...,동일하이빌파크포레,동일하이빌파크포레,인천광역시,연수구,동춘동,4,54,1134,2818510500,103.333333


In [45]:
final_result_addr_all.columns

Index(['kb_unit_type_id', 'kb_apartment_id', 'net_area', 'number_of_units',
       'rent_ratio', 'kb_price_growth', 'kb_rent_growth', 'x', 'y',
       'number_of_parking', 'academy', 'harmful_facility', 'subway', 'culture',
       'department_store', 'mart', 'cafe', 'convenience', 'hospital',
       'pharmacy', 'facility_info', 'close_apt_price_growth',
       'close_apt_rent_growth', 'kb_rent_growth_24', 'kb_rent_growth_36',
       'price_per_size', 'rent_per_size', '빌딩_5000', 'subway_index',
       'min_subway_distance', 'max_subway_distance', 'mean_subway_distance',
       '상가_100', '상가_200', '스타벅스_2500', '학교_3500', '학교_700', '도서관_3500',
       'regist_for_earthquake', '우체국_5000', '빌라_3500', 'IC_6000', '골프_4000',
       '오피스텔_6000', '주상복합_5000', '개발지구_6500', '강남역_소요시간', '시청역_소요시간',
       '여의도역_소요시간', '전기자동차 충전소_6500', '와인_4000', '카셰어링_3000', '가산디지털단지역_소요시간',
       '성수역_소요시간', '판교역_소요시간', 'cnt_nm', 'distance', 'Count',
       'number_of_units_total', 'number_of_dongs', 'floor_highe

In [46]:
# col_list = ['kb_unit_type_id', 'kb_apartment_id', 'number_of_units_total','net_area', 'number_of_units', 'age', 
#        'subway_index', 'min_subway_distance', 
#        'culture','department_store', 'mart', 'cafe', 'convenience',
#        '빌딩_5000', '상가_100', '상가_200',
#        '강남역_소요시간', '시청역_소요시간', '여의도역_소요시간',
#        'number_of_rooms', 'number_of_bathrooms',   'close_apt_price_growth' , '개발지구_6000', 'price_per_size', 'rent_per_size']
col_list = ['kb_unit_type_id', 'kb_apartment_id', 'x', 'y', 'number_of_units_total','net_area', 'number_of_units', 'age', 
       'subway_index', 'min_subway_distance', 
       'culture','department_store', 'mart', 'cafe', 'convenience', 'hospital', 'pharmacy',
       'close_apt_price_growth', 'close_apt_rent_growth', 'kb_rent_growth_24','kb_rent_growth_36',
       '빌딩_5000', '상가_100', '상가_200',
        '스타벅스_2500', '학교_3500', '학교_700', '도서관_3500',
       '우체국_5000', '빌라_3500', 'IC_6000',
       '오피스텔_6000', '주상복합_5000', '개발지구_6500',
       '강남역_소요시간', '시청역_소요시간', '여의도역_소요시간', '성수역_소요시간', '판교역_소요시간', 'avg_trans_time',
       'number_of_dongs', 'floor_highest_dong', 'floor_lowest_dong',
       'number_of_rooms', 'number_of_bathrooms',  '전기자동차 충전소_6500', '와인_4000', '카셰어링_3000', '골프_4000', 'rebuild', 'rent_per_size', 'price_per_size']

In [48]:
res_dt_1.drop(columns=['kb_apartment_id', 'kb_unit_type_id']).corr()["price_per_size"].sort_values(ascending = False)

price_per_size            1.000000
rent_per_size             0.919913
골프_4000                   0.787220
빌딩_5000                   0.773535
스타벅스_2500                 0.743747
와인_4000                   0.736624
IC_6000                   0.653777
department_store          0.652793
카셰어링_3000                 0.651809
전기자동차 충전소_6500            0.629929
mart                      0.620360
우체국_5000                  0.605185
개발지구_6500                 0.591904
hospital                  0.568204
학교_3500                   0.526412
culture                   0.522573
오피스텔_6000                 0.521638
주상복합_5000                 0.515277
도서관_3500                  0.496287
subway_index              0.461546
학교_700                    0.402500
빌라_3500                   0.358420
cafe                      0.320555
상가_200                    0.308919
상가_100                    0.294897
pharmacy                  0.288652
floor_highest_dong        0.266428
number_of_units_total     0.238832
number_of_dongs     

In [49]:
res_dt_1.drop(columns=['kb_apartment_id', 'kb_unit_type_id']).corr()["rent_per_size"].sort_values(ascending = False)

rent_per_size             1.000000
price_per_size            0.919913
골프_4000                   0.736760
빌딩_5000                   0.715449
스타벅스_2500                 0.707713
와인_4000                   0.687990
department_store          0.617759
카셰어링_3000                 0.615862
IC_6000                   0.609198
전기자동차 충전소_6500            0.594065
mart                      0.585999
우체국_5000                  0.558251
개발지구_6500                 0.549964
hospital                  0.529539
culture                   0.507210
학교_3500                   0.496425
오피스텔_6000                 0.483440
주상복합_5000                 0.476618
도서관_3500                  0.460478
subway_index              0.433035
floor_highest_dong        0.376128
학교_700                    0.375983
빌라_3500                   0.330787
cafe                      0.322180
상가_200                    0.278685
pharmacy                  0.278613
상가_100                    0.264121
kb_rent_growth_36         0.253969
number_of_units_tota

In [51]:
# 전세가 산정 학습 데이터 생성
non_predict_value = 'price_per_size'
res_dt_2 = res_dt_1.drop(columns=[non_predict_value])
res_dt_2

,kb_unit_type_id,kb_apartment_id,x,y,number_of_units_total,net_area,number_of_units,age,subway_index,min_subway_distance,...,floor_highest_dong,floor_lowest_dong,number_of_rooms,number_of_bathrooms,전기자동차 충전소_6500,와인_4000,카셰어링_3000,골프_4000,rebuild,rent_per_size
0,1,1,126.968160,37.591500,126,83.61,42,34.0,0.000000,0,...,3,3,3,1,853,245,47,161,1,1858.293207
1,2,1,126.968160,37.591500,126,132.39,20,34.0,0.000000,0,...,3,3,4,2,853,245,47,161,1,1423.292885
2,3,1,126.968160,37.591500,126,145.73,10,34.0,0.000000,0,...,3,3,5,2,853,245,47,161,1,1361.058807
3,4,2,126.967381,37.586360,60,84.51,20,22.0,0.000000,0,...,5,5,3,2,869,287,66,175,0,2347.025204
4,5,2,126.967381,37.586360,60,129.76,20,22.0,0.000000,0,...,5,5,4,2,869,287,66,175,0,1783.330379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25782,66418,21007,126.848279,37.371110,968,84.98,95,3.0,0.000000,0,...,25,24,3,2,289,8,4,42,0,1731.082990
25783,66419,21008,126.958711,37.500384,423,59.51,147,19.0,2.156986,637,...,17,9,3,1,983,263,127,308,0,3221.904386
25784,66420,21008,126.958711,37.500384,423,84.99,276,19.0,2.156986,637,...,17,9,3,2,983,263,127,308,0,2819.971908
25785,66459,21013,126.657820,37.412493,281,59.97,181,3.0,0.000000,0,...,22,22,3,2,387,38,26,123,0,1819.091296


In [54]:
train_data = res_dt_2.drop(columns=["kb_unit_type_id", "kb_apartment_id", res_dt_2.columns[-1]])
train_data

,x,y,number_of_units_total,net_area,number_of_units,age,subway_index,min_subway_distance,culture,department_store,...,number_of_dongs,floor_highest_dong,floor_lowest_dong,number_of_rooms,number_of_bathrooms,전기자동차 충전소_6500,와인_4000,카셰어링_3000,골프_4000,rebuild
0,126.968160,37.591500,126,83.61,42,34.0,0.000000,0,423.0,10.0,...,9,3,3,3,1,853,245,47,161,1
1,126.968160,37.591500,126,132.39,20,34.0,0.000000,0,423.0,10.0,...,9,3,3,4,2,853,245,47,161,1
2,126.968160,37.591500,126,145.73,10,34.0,0.000000,0,423.0,10.0,...,9,3,3,5,2,853,245,47,161,1
3,126.967381,37.586360,60,84.51,20,22.0,0.000000,0,460.0,11.0,...,4,5,5,3,2,869,287,66,175,0
4,126.967381,37.586360,60,129.76,20,22.0,0.000000,0,460.0,11.0,...,4,5,5,4,2,869,287,66,175,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25782,126.848279,37.371110,968,84.98,95,3.0,0.000000,0,1.0,3.0,...,11,25,24,3,2,289,8,4,42,0
25783,126.958711,37.500384,423,59.51,147,19.0,2.156986,637,37.0,19.0,...,7,17,9,3,1,983,263,127,308,0
25784,126.958711,37.500384,423,84.99,276,19.0,2.156986,637,37.0,19.0,...,7,17,9,3,2,983,263,127,308,0
25785,126.657820,37.412493,281,59.97,181,3.0,0.000000,0,28.0,1.0,...,6,22,22,3,2,387,38,26,123,0


In [55]:
price_data = pd.DataFrame(res_dt_2[res_dt_2.columns[-1]], columns=[res_dt_2.columns[-1]])
price_data

,rent_per_size
0,1858.293207
1,1423.292885
2,1361.058807
3,2347.025204
4,1783.330379
...,...
25782,1731.082990
25783,3221.904386
25784,2819.971908
25785,1819.091296


In [56]:
min_max_scaler1  = MinMaxScaler()

min_max_scaler1.fit(train_data)

train_data_scaled = min_max_scaler1.fit_transform(train_data)
train_data_scaled = pd.DataFrame(train_data_scaled, columns = train_data.columns)
train_data_scaled

,x,y,number_of_units_total,net_area,number_of_units,age,subway_index,min_subway_distance,culture,department_store,...,number_of_dongs,floor_highest_dong,floor_lowest_dong,number_of_rooms,number_of_bathrooms,전기자동차 충전소_6500,와인_4000,카셰어링_3000,골프_4000,rebuild
0,0.413988,0.590238,0.008034,0.282542,0.007991,0.596154,0.000000,0.000000,0.589958,0.50,...,0.066116,0.000000,0.066667,0.333333,0.000000,0.423788,0.181347,0.179389,0.114613,1.0
1,0.413988,0.590238,0.008034,0.484246,0.003703,0.596154,0.000000,0.000000,0.589958,0.50,...,0.066116,0.000000,0.066667,0.500000,0.333333,0.423788,0.181347,0.179389,0.114613,1.0
2,0.413988,0.590238,0.008034,0.539406,0.001754,0.596154,0.000000,0.000000,0.589958,0.50,...,0.066116,0.000000,0.066667,0.666667,0.333333,0.423788,0.181347,0.179389,0.114613,1.0
3,0.413329,0.585456,0.001057,0.286264,0.003703,0.365385,0.000000,0.000000,0.641562,0.55,...,0.024793,0.037037,0.111111,0.333333,0.333333,0.431784,0.212435,0.251908,0.124642,0.0
4,0.413329,0.585456,0.001057,0.473371,0.003703,0.365385,0.000000,0.000000,0.641562,0.55,...,0.024793,0.037037,0.111111,0.500000,0.333333,0.431784,0.212435,0.251908,0.124642,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25782,0.312534,0.385230,0.097040,0.288207,0.018320,0.000000,0.000000,0.000000,0.001395,0.15,...,0.082645,0.407407,0.533333,0.333333,0.333333,0.141929,0.005922,0.015267,0.029370,0.0
25783,0.405991,0.505481,0.039429,0.182890,0.028454,0.307692,0.177857,0.637638,0.051604,0.95,...,0.049587,0.259259,0.200000,0.333333,0.000000,0.488756,0.194671,0.484733,0.219914,0.0
25784,0.405991,0.505481,0.039429,0.288248,0.053596,0.307692,0.177857,0.637638,0.051604,0.95,...,0.049587,0.259259,0.200000,0.333333,0.333333,0.488756,0.194671,0.484733,0.219914,0.0
25785,0.151350,0.423724,0.024419,0.184792,0.035081,0.000000,0.000000,0.000000,0.039052,0.05,...,0.041322,0.351852,0.488889,0.333333,0.333333,0.190905,0.028127,0.099237,0.087393,0.0


In [58]:
dump(min_max_scaler1, open(dir+'\\data_scaler_rent.pkl', 'wb'))

In [59]:
min_max_scaler2  = MinMaxScaler()
min_max_scaler2.fit(price_data)

price_data_scaled = min_max_scaler2.fit_transform(price_data)
price_data_scaled = pd.DataFrame(price_data_scaled, columns =[res_dt_2.columns[-1] + '_scaled'])
price_data_scaled

,rent_per_size_scaled
0,0.188648
1,0.139891
2,0.132916
3,0.243428
4,0.180246
...,...
25782,0.174390
25783,0.341490
25784,0.296439
25785,0.184254


In [60]:
dump(min_max_scaler2, open(dir+'\\price_scaler_rent.pkl', 'wb'))

In [61]:
# std_scaler  = StandardScaler()
# res_dt_scale = std_scaler.fit_transform(res_dt)
# res_dt_scale = pd.DataFrame(res_dt_scale, columns = res_dt.columns)
# res_dt_scale

In [62]:
# res_dt_scale.drop(columns=['kb_apartment_id', 'kb_unit_type_id']).corr()["gap"].sort_values(ascending = False)

In [63]:
# X_train, X_test, y_train, y_test = train_test_split(train_data_scaled, price_data_scaled ,test_size=0.25)

In [64]:
# xgb_model = xgboost.XGBRegressor(n_estimators=5000, 
#                                  learning_rate=0.2,
#                                  rate_drop=0.1,
#                                  max_depth =  10,
#                                 sample_type = 'uniform',
#                                 booster = 'dart')
# # xgb_model.fit(res_dt.drop(columns=["rent_per_size"]), res_dt["rent_per_size"])
# xgb_model.fit(train_data_scaled, price_data_scaled)

In [65]:
param_test2 = {'learning_rate' : [0.2, 0.18, 0.185, 0.19],
               'n_estimators' : [ 3000, 5000],
               'max_depth' : [10, 20, 30],
              'subsample' : [0.5]}
 
kf = KFold(n_splits=2)

gsearch2 = GridSearchCV(estimator = xgboost.XGBRegressor(
#                                                   min_child_weight=1, 
#                                                   gamma=0, 
#                                                   colsample_bytree=0.8,
#                                                   objective= 'binary:logistic', 
                                                  scale_pos_weight=1,
                                                  sample_type = 'uniform',
                                                  max_features = 'auto',
#                                                   rate_drop = 0.1,
                                                  booster = 'dart'),
#                         param_grid = param_test2, scoring='neg_mean_absolute_error',n_jobs=6, cv=kf, verbose=10)
                        param_grid = param_test2, scoring='neg_mean_squared_error', n_jobs=6, cv=kf, verbose=10)

# gsearch2.fit(X_train, y_train)
gsearch2.fit(train_data_scaled, price_data_scaled)
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

Fitting 2 folds for each of 24 candidates, totalling 48 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed: 462.4min
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed: 892.8min
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed: 2318.7min
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed: 3323.3min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed: 5005.9min
[Parallel(n_jobs=6)]: Done  42 out of  48 | elapsed: 6964.1min remaining: 994.9min
[Parallel(n_jobs=6)]: Done  48 out of  48 | elapsed: 7858.2min finished


[00:50:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




({'mean_fit_time': array([27803.66601634, 73848.76597226, 27718.02612352, 77433.03912103,
         25660.31718993, 85617.09083402, 31437.05847466, 93953.69788253,
         33766.74468136, 90718.14636374, 34113.65267801, 92306.86101305,
         30767.60550773, 94241.7560358 , 35562.16384947, 89850.05169821,
         34477.99488306, 86697.28761172, 31484.70110917, 85514.78159022,
         31453.27567971, 74431.73481536, 30330.00518906, 53664.52343023]),
  'std_fit_time': array([ 7.08404922, 43.31094468, 10.92078424, 17.34937894, 31.13021994,
         70.16389692, 26.4699949 , 98.67243636, 24.16604519, 64.21000314,
         37.68296599, 39.29602206, 13.85094488, 99.75213861, 57.07689083,
         54.77988791, 42.83260441, 73.26300383, 51.84729528, 50.98410559,
         24.45607436, 42.19112849, 47.06110108, 45.51773036]),
  'mean_score_time': array([32.88751781, 68.45145249, 33.56021821, 92.4446789 , 29.83967197,
         76.00258207, 37.60742056, 73.10442996, 43.09590101, 57.79937351,
 

In [66]:
gsearch2.best_params_

{'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 5000, 'subsample': 0.5}

In [90]:
train_data_scaled = train_data_scaled.drop(columns=['predict_xgb_rent_per_size', '100-MAPE'])

In [67]:
xgb_model = xgboost.XGBRegressor(n_estimators=5000, 
                                 learning_rate=0.2,
                                 rate_drop=0.1,
                                 max_depth =  10,
                                  subsample = 0.5,
                                sample_type = 'uniform',
                                booster = 'dart')
# xgb_model.fit(res_dt.drop(columns=["rent_per_size"]), res_dt["rent_per_size"])
xgb_model.fit(train_data_scaled, price_data_scaled)

ERROR! Session/line number was not unique in database. History logging moved to new session 2517


KeyboardInterrupt: 

In [ ]:
imp = (xgb_model.feature_importances_)

res_ft = pd.DataFrame({'col_nm' : train_data_scaled.columns.values, 'feature_importance' : imp})
res_ft = res_ft.sort_values(by=["feature_importance"], ascending = False)
res_ft

In [ ]:
# res_dt = res_dt.drop(columns=["predict_RDF_price_per_size", "100-MAPE_RDF"])
train_data_scaled["predict_xgb_rent_per_size"] = xgb_model.predict(train_data_scaled)
train_data_scaled["100-MAPE"] = 100 - ((abs(price_data_scaled[price_data_scaled.columns[0]] - train_data_scaled["predict_xgb_rent_per_size"]) / price_data_scaled[price_data_scaled.columns[0]]) * 100)
train_data_scaled["100-MAPE"].mean()

In [ ]:
train_data_scaled[train_data_scaled["100-MAPE"] == -np.inf]

In [ ]:
train_data_scaled[train_data_scaled["100-MAPE"] > 0]["100-MAPE"].mean()

In [89]:
train_data_scaled[train_data_scaled["100-MAPE"] > 0]["100-MAPE"].mean()

95.20228745844248

In [80]:
train_data_scaled[train_data_scaled["100-MAPE"] > 0]["100-MAPE"].mean()

95.61117890984491

In [70]:
train_data_scaled[train_data_scaled["100-MAPE"] > 0]["100-MAPE"].mean()

95.57737474626681

In [ ]:
# train_data_scaled = train_data_scaled.drop(columns=['predict_xgb_rent_per_size', '100-MAPE'])

In [71]:
predict_result_rent1 = min_max_scaler1.inverse_transform(train_data_scaled)
predict_result_rent1 = pd.DataFrame(predict_result_rent1, columns = train_data_scaled.columns)
predict_result_rent1

ValueError: Input contains infinity or a value too large for dtype('float64').

In [ ]:
predict_result_rent2 = pd.DataFrame(price_data_scaled["predict_xgb_rent_per_size"], columns = ["predict_xgb_rent_per_size"])
predict_result_rent2 = min_max_scaler2.inverse_transform(predict_result_rent2)
predict_result_rent2 = pd.DataFrame(predict_result_rent2, columns = ["predict_xgb_rent_per_size"])
predict_result_rent2

In [ ]:
predict_result_rent3 = pd.concat([price_data, predict_result_rent2], axis = 1)
# predict_result_rent3 = predict_result_rent3[predict_result_rent3['']]
predict_result_rent3['결과차이(평당가/원)'] =(abs(predict_result_rent3['rent_per_size'] - predict_result_rent3['predict_xgb_rent_per_size']))
predict_result_rent3["100-MAPE"] = 100 - ((abs(predict_result_rent3['rent_per_size'] - predict_result_rent3["predict_xgb_rent_per_size"]) / predict_result_rent3['rent_per_size']) * 100)
predict_result_rent3

In [ ]:
predict_result_rent3['100-MAPE'].value_counts()

In [ ]:
predict_result_rent3['100-MAPE'].describe()

In [ ]:
predict_result_rent3[predict_result_rent3['100-MAPE'] < 90]

In [ ]:
predict_result_rent3[predict_result_rent3['100-MAPE'] < 80]

In [ ]:
predict_result_rent3[predict_result_rent3['100-MAPE'] < 50]

In [ ]:
predict_result_rent3['100-MAPE'].mean()

In [ ]:
predict_result_rent = pd.concat([std_id, predict_result_rent1, predict_result_rent3], axis = 1)
predict_result_rent

In [ ]:
predict_result_rent["predict_kb_rent"] = (predict_result_rent["predict_xgb_rent_per_size"] / 3.305785) * predict_result_rent['net_area']


 (snu_data["kb_rent"] / snu_data["net_area"]) * 3.305785


In [ ]:
predict_result_rent.to_csv(dir + '\\predict_result_rent.csv', index=False, encoding='utf8')

In [ ]:
import pickle

import datetime
# import timedelta
# 오늘날짜 기준
dt = datetime.datetime.now()
fl_dt = str(dt.year) + str(dt.month) + str(dt.day)

pickle.dump(xgb_model, open('xgb_model_수도권v0.89(매매가)_pkl_' + fl_dt, 'wb'))

In [ ]:
import joblib

import datetime
# import timedelta
# 오늘날짜 기준
dt = datetime.datetime.now()
fl_dt = str(dt.year) + str(dt.month) + str(dt.day)

joblib.dump(xgb_model, open('xgb_model_수도권(매매가)_joblib_' + fl_dt, 'wb'))

In [ ]:
load_md1 = pickle.load(open('xgb_model_수도권(매매가)_pkl_' + fl_dt,'rb'))
load_md1

In [ ]:
load_md2 = pickle.load(open('xgb_model_수도권(매매가)_joblib_' + fl_dt,'rb'))
load_md2